In [ ]:
import json
from rapidfuzz import fuzz
import itertools

In [ ]:
with open("your_output_file_here", "r") as f:
    data = json.load(f)
results = data["results"]

In [ ]:
def compare_material(gt_mats, pred_mats, verbose=False):
    if verbose:
        print("COMPARING MATERIALS")
        print("gt_mats: ", gt_mats)
        print("pred_mats: ", pred_mats)
    TPs = 0
    for gt_mat in gt_mats:
        for pred_mat in pred_mats:
            if fuzz.partial_ratio(gt_mat, pred_mat) > 90:
                TPs += 1
                pred_mats.remove(pred_mat)
                if verbose:
                    print("MATCH ", gt_mat, pred_mat)

    FPs = len(pred_mats)
    FNs = len(gt_mats) - TPs

    n_items = TPs + FPs + FNs
    if n_items > 0:
        acc = float(TPs) / float(n_items)
        if TPs + FPs > 0:
            prec = TPs/(TPs + FPs)
        else:
            prec = 1.0
        if TPs + FNs > 0:
            rec = TPs/(TPs + FNs)
        else:
            rec = 1.0
        if verbose:
            print("ACC: ",acc)
            print("PREC: ",prec)
            print("REC: ",rec)
        return acc, prec, rec
    else:
        return 1.0, 1.0, 1.0

In [ ]:
def compare_tols(gt_tols, pred_tols, verbose=False):
    if verbose:
        print("COMPARING TOLS")
        print("GT: ", gt_tols)
        print("PRED: ", pred_tols)
    TPs = 0
    FPs = 0
    FNs = 0
    if gt_tols["char1"] == pred_tols["char1"]:
        TPs += 1
    else:
        if len(pred_tols["char1"]) > 0:
            FPs += 1
        else:
            FNs += 1
    if gt_tols["char2"] == pred_tols["char2"]:
        TPs += 1
    else:
        if len(pred_tols["char2"]) > 0:
            FPs += 1
        else:
            FNs += 1
    acc = TPs/2
    if TPs + FPs > 0:
        prec = TPs/(TPs + FPs)
    else:
        prec = 1.0
    if TPs + FNs > 0:
        rec = TPs/(TPs + FNs)
    else:
        rec = 1.0
    if verbose:
        print("ACC: ",acc)
        print("PREC: ",prec)
        print("REC: ",rec)
    return acc, prec, rec

In [ ]:
def compare_names(gt_name, pred_name, verbose=False):
    acc = fuzz.partial_ratio(gt_name.lower(), pred_name.lower())
    if verbose:
        print("COMPARING NAMES")
        print("GT: ",gt_name)
        print("PRED: ",pred_name)
        print("RESULT: ", acc)
    if acc > 90:
        return 1.0
    else:
        return 0

In [ ]:
def compare_strings(s1: str, s2: str, ratio):
    s1 = s1.replace(" ", "")
    s2 = s2.replace(" ", "")
    return fuzz.partial_ratio(s1, s2) > ratio
def compare_surfaces(gt_surfs, pred_surfs, verbose=False):
    if verbose:
        print("COMPARING SURFACES ")
        print("GT: ", gt_surfs)
        print("PRED: ", pred_surfs)
    TPs = 0
    for gt_surf in gt_surfs:
        for pred_surf in pred_surfs:
            if verbose:
                print(gt_surf, pred_surf)
            if compare_strings(gt_surf, pred_surf, 80):
                if verbose:
                    print("MATCH ")
                TPs += 1
                pred_surfs.remove(pred_surf)
                break

    FPs = len(pred_surfs)
    FNs = len(gt_surfs) - TPs

    if verbose:
        print("TPs:", TPs, "FPs:", FPs, "FNs:", FNs)

    n_items = TPs + FPs + FNs
    if n_items > 0:
        acc = float(TPs) / float(n_items)
        if TPs + FPs > 0:
            prec = TPs/(TPs + FPs)
        else:
            prec = 1.0
        if TPs + FNs > 0:
            rec = TPs/(TPs + FNs)
        else:
            rec = 1.0
        if verbose:
            print("ACC: ",acc)
            print("PREC: ",prec)
            print("REC: ",rec)
        return acc, prec, rec
    else:
        return 1.0, 1.0, 1.0

In [ ]:
def compare_gdts(gt_gdts, pred_gdts, verbose=False):
    if verbose:
        print("COMPARING GTDS")
        print("GT: ", gt_gdts)
        print("PRED: ", pred_gdts)
    gt_gdts_condensed = []
    for gt_gdt in gt_gdts:
        gt_gdts_condensed.append(gt_gdt["name"]+"_"+str(gt_gdt["runout"]))

    pred_gdts_condensed = []
    for pred_gdt in pred_gdts:
        pred_gdts_condensed.append(pred_gdt["name"]+"_"+str(pred_gdt["runout"]))

    TPs = 0
    for gt_gdt in gt_gdts_condensed:
        for pred_gdt in pred_gdts_condensed:
            if verbose:
                print(gt_gdt, pred_gdt)
            if gt_gdt == pred_gdt:
                if verbose:
                    print("MATCH")
                TPs += 1
                pred_gdts_condensed.remove(pred_gdt)
                break

    FPs = len(pred_gdts_condensed)
    FNs = len(gt_gdts_condensed) - TPs

    if verbose:
        print("TPs:", TPs, "FPs:", FPs, "FNs:", FNs)

    n_items = TPs + FPs + FNs
    if n_items > 0:
        acc = float(TPs) / float(n_items)
        if TPs + FPs > 0:
            prec = TPs/(TPs + FPs)
        else:
            prec = 1.0
        if TPs + FNs > 0:
            rec = TPs/(TPs + FNs)
        else:
            rec = 1.0
        if verbose:
            print("ACC: ",acc)
            print("PREC: ",prec)
            print("REC: ",rec)
        return acc, prec, rec
    else:
        return 1.0, 1.0, 1.0

In [ ]:
def compare_threads(gt_threads, pred_threads, verbose=False):
    if verbose:
        print("COMPARING THREADS")
        print("GT: ", gt_threads)
        print("PRED: ", pred_threads)
    TPs = 0
    for gt_thread in gt_threads:
        for pred_thread in pred_threads:
            if verbose:
                print(gt_thread, pred_thread)

            if compare_strings(gt_thread, pred_thread, 80):
                if verbose:
                    print("MATCH")
                TPs += 1
                pred_threads.remove(pred_thread)
                break

    FPs = len(pred_threads)
    FNs = len(gt_threads) - TPs
    if verbose:
        print("TPs:", TPs, "FPs:", FPs, "FNs:", FNs)

    n_items = TPs + FPs + FNs
    if n_items > 0:
        acc = float(TPs) / float(n_items)
        if TPs + FPs > 0:
            prec = TPs/(TPs + FPs)
        else:
            prec = 1.0
        if TPs + FNs > 0:
            rec = TPs/(TPs + FNs)
        else:
            rec = 1.0
        if verbose:
            print("ACC: ",acc)
            print("PREC: ",prec)
            print("REC: ",rec)
        return acc, prec, rec
    else:
        return 1.0, 1.0, 1.0

In [ ]:
def compare_outer_measures(gt_measures, pred_measures, verbose = False):
    if verbose:
        print("COMPARING OUTER MEASURES")
        print("GT: ", gt_measures)
        print("PRED: ", pred_measures)
    TPs = 0
    for gt_measure in gt_measures:
        for pred_measure in pred_measures:
            if gt_measure == pred_measure:
                TPs += 1
                pred_measures.remove(pred_measure)
                break

    FPs = len(pred_measures)
    FNs = len(gt_measures) - TPs
    n_items = TPs + FPs + FNs
    if n_items > 0:
        acc = float(TPs) / float(n_items)
        if TPs + FPs > 0:
            prec = TPs/(TPs + FPs)
        else:
            prec = 1.0
        if TPs + FNs > 0:
            rec = TPs/(TPs + FNs)
        else:
            rec = 1.0
        if verbose:
            print("ACC: ",acc)
            print("PREC: ",prec)
            print("REC: ",rec)
        return acc, prec, rec
    else:
        return 1.0, 1.0, 1.0

In [ ]:
def get_measure_distance(measures1, measures2):
    x1, y1, z1 = measures1
    x2, y2, z2 = measures2

    dists = []

    for dim1, dim2 in zip([x1,y1,z1], [x2,y2,z2]):
        if dim1 == 0 or dim2 == 0:
            dist = max(dim1, dim2)
        elif dim1 > dim2:
            dist = dim1 / dim2
        else:
            dist = dim2 / dim1

        dists.append(dist)

    return sum(dists)

def compare_outer_measure_distance(gt_measures, pred_measures, verbose = False):
    if verbose:
        print("COMPARING OUTER MEASURE DISTANCE")
        print("GT: ", gt_measures)
        print("PRED: ", pred_measures)

    x,y,z = gt_measures

    best_distance = np.inf

    while len(pred_measures) < 3:
        pred_measures.append(0)

    for permutation in list(itertools.permutations([x,y,z])):
        print("Checking distance between ", permutation, pred_measures, "...")
        distance = get_measure_distance(permutation, pred_measures)
        print("Distance: ", distance)
        if distance < best_distance:
            print("New best distance")
            best_distance = distance

    print("Best distance: ", best_distance)
    return best_distance


In [ ]:
mat_sims = []
tol_sims = []
name_sims = []
surface_sims = []
gdt_sims = []
thread_sims = []
measure_sims = []
measure_distances = []

for result in results:
    print("Calling function...")
    gt = result["ground_truth"]
    vlm_response = json.loads(result["vlm_reponse"])

    mat_sims.append(compare_material(gt["material"], vlm_response["material"]))
    tol_sims.append(compare_tols(gt["general_tolerances"], vlm_response["general_tolerances"]))
    name_sims.append(compare_names(gt["name"], vlm_response["name"]))
    surface_sims.append(compare_surfaces(gt["surfaces"], vlm_response["surfaces"]))
    gdt_sims.append(compare_gdts(gt["gdts"], vlm_response["gdts"]))
    thread_sims.append(compare_threads(gt["threads"], vlm_response["threads"]))
    measure_sims.append(compare_outer_measures(gt["outer_measures"], vlm_response["outer_measures"]))
    measure_distances.append(compare_outer_measure_distance(gt["outer_measures"], vlm_response["outer_measures"], True))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
print(np.mean(np.asarray(name_sims)))

In [ ]:
sims = [
    mat_sims,
    tol_sims,
    name_sims,
    surface_sims,
    gdt_sims,
    thread_sims,
    measure_sims,
    measure_distances
]

names = [
    "mat_sims",
    "tol_sims",
    "name_sims",
    "surface_sims",
    "gdt_sims",
    "thread_sims",
    "measure_sims",
    "measure_distances"
]

for sim, name in zip(sims, names):
    sim = np.asarray(sim)
    avg = sim.mean()
    mode = np.median(sim)
    plt.figure()
    plt.title(name + " " + str(avg))
    plt.hist(sim, bins = 20)
    plt.vlines(avg, 0, 50, colors="r")
    plt.vlines(mode, 0, 50, colors="g")
    plt.show()

In [ ]:
mat_sims = {
    "accs": [],
    "prec": [],
    "rec": []
}
tol_sims = {
    "accs": [],
    "prec": [],
    "rec": []
}
name_sims = []
surface_sims = {
    "accs": [],
    "prec": [],
    "rec": []
}
gdt_sims = {
    "accs": [],
    "prec": [],
    "rec": []
}
thread_sims = {
    "accs": [],
    "prec": [],
    "rec": []
}
measure_sims = {
    "accs": [],
    "prec": [],
    "rec": []
}
measure_distances = []

for result in results:
    print("Calling function...")
    gt = result["ground_truth"]
    vlm_response = json.loads(result["vlm_reponse"])

    helper = [
        (mat_sims, compare_material, "material", False),
        (tol_sims, compare_tols, "general_tolerances", False),
        (surface_sims, compare_surfaces,"surfaces", False),
        (gdt_sims, compare_gdts, "gdts", False),
        (thread_sims, compare_threads, "threads", False),
        (measure_sims, compare_outer_measures,"outer_measures", False)
              ]

    for store_dict, comparison_function, name, verbose in helper:
        acc, prec, rec = comparison_function(gt[name], vlm_response[name], verbose)
        store_dict["accs"].append(acc)
        store_dict["prec"].append(prec)
        store_dict["rec"].append(rec)

    name_sims.append(compare_names(gt["name"], vlm_response["name"]))
    measure_distances.append(compare_outer_measure_distance(gt["outer_measures"], vlm_response["outer_measures"]))

In [ ]:
sims = [
    mat_sims,
    tol_sims,
    # name_sims,
    surface_sims,
    gdt_sims,
    thread_sims,
    measure_sims,
    # measure_distances
]

names = [
    "mat_sims",
    "tol_sims",
    # "name_sims",
    "surface_sims",
    "gdt_sims",
    "thread_sims",
    "measure_sims",
    # "measure_distances"
]

for sim, name in zip(sims, names):
    for measure in ["accs", "prec", "rec"]:
        data = np.asarray(sim[measure])
        avg = data.mean()
        mode = np.median(data)
        plt.figure()
        plt.title(name + f" {measure} " + str(avg))
        plt.hist(data, bins = 20)
        plt.vlines(avg, 0, 50, colors="r")
        plt.vlines(mode, 0, 50, colors="g")
        plt.show()